## 作業目標: 了解斷詞演算法的背後計算

### 根據課程講述的內容, 請計算出下列剩餘所有情況的
若有一個人連續觀察到三天水草都是乾燥的(Dry), 則這三天的天氣機率為何？(可參考講義第13頁)
(Hint: 共有8種可能機率)

```python
states = ('sunny', 'rainy')
observations = ('dry', 'damp', 'soggy')
start_probability = {'sunny': 0.4, 'rainy': 0.6}
transition_probability = {'sunny':{'sunny':0.6, 'rainy':0.4},
                          'rainy': {'sunny':0.3, 'rainy':0.7}}
emission_probatility = {'sunny': {'dry':0.6, 'damp':0.3, 'soggy':0.1},
                        'rainy': {'dry':0.1, 'damp':0.4, 'soggy':0.5}}
```

```
觀察狀態 = 'dry', 'dry', 'dry'
Sunny, Sunny, Sunny: 0.4*(0.6)*0.6*(0.6)*0.6*(0.6) = 0.031104
Rainy, Sunny, Sunny: 0.6*(0.1)*0.3*(0.6)*0.6*(0.6) = 0.003888
###<your answers>###

最大機率為: Sunny, Sunny, Sunny
```

Toy [example](https://www.cis.upenn.edu/~cis262/notes/Example-Viterbi-DNA.pdf) from CIS

### 根據上述條件, 寫出Viterbi應用程式

In [1]:
observations = ('dry', 'dry', 'dry') #實際上觀察到的狀態為dry, dry, dry
states = ('sunny', 'rainy')
start_probability = {'sunny': 0.4, 'rainy': 0.6}
transition_probability = {'sunny': {'sunny': 0.6, 'rainy': 0.4},
                          'rainy': {'sunny': 0.3, 'rainy': 0.7}}
emission_probatility = {'sunny': {'dry': 0.6, 'damp': 0.3, 'soggy': 0.1},
                        'rainy': {'dry': 0.1, 'damp': 0.4, 'soggy': 0.5}}

In [4]:
def viterbi(observations, states, start_p, transition_p, emission_p):
    V = [{}]
    path = {}

    # Initialize base cases (t == 0)
    for y in states:
        V[0][y] = start_p[y] * emission_p[y][observations[0]]
        path[y] = [y]
        
    # Run Viterbi for t > 0
    for t in range(1, len(observations)):
        V.append({})
        newpath = {}
        for current_state in states:
            (prob, state) = max(
                [(V[t-1][pre_state]*transition_p[pre_state][current_state]*emission_p[current_state][observations[t]], 
                  pre_state) 
                 for pre_state in states])
            V[t][current_state] = prob
            newpath[current_state] = path[state] + [current_state]

        # Don't need to remember the old paths
        path = newpath
    
    (prob, state) = max([(V[len(observations)-1][final_state], final_state) for final_state in states])
    return (prob, path[state])

In [ ]:
import math

def viterbi_log(observations, states, start_p, transition_p, emission_p):
    for key in start_p:
        start_p[key] = math.log2(start_p.get(key))
    for key in transition_p:
        dictionary = transition_p.get(key)
        for k in dictionary:
            dictionary[k] = math.log2(dictionary.get(k))
    for key in emission_p:
        dictionary = emission_p.get(key)
        for k in dictionary:
            dictionary[k] = math.log2(dictionary.get(k))
    V = [{}]
    path = {}

    # Initialize base cases (t == 0)
    for y in states:
        V[0][y] = start_p[y] + emission_p[y][observations[0]]
        path[y] = [y]
        
    # Run Viterbi for t > 0
    for t in range(1, len(observations)):
        V.append({})
        newpath = {}
        for current_state in states:
            (prob, state) = max(
                [(V[t-1][pre_state]+transition_p[pre_state][current_state]+emission_p[current_state][observations[t]], 
                  pre_state) 
                 for pre_state in states])
            V[t][current_state] = prob
            newpath[current_state] = path[state] + [current_state]

        # Don't need to remember the old paths
        path = newpath
    
    (prob, state) = max([(V[len(observations)-1][final_state], final_state) for final_state in states])
    return (prob, path[state])

In [5]:
result = viterbi(observations,
                 states,
                 start_probability,
                 transition_probability,
                 emission_probatility)

In [7]:
result

(0.031103999999999993, ['sunny', 'sunny', 'sunny'])